In [6]:
import xgboost as xgb
import matplotlib.pyplot as plt

# ---------- CONFIG ----------
MODEL_PATH = "model.json"
MAX_IMPORTANT_FEATURES = 20   # para el gráfico de importancia
TOP_FEATURES_FOR_TREES = 5    # cuántas vars usar para elegir árboles
FIGSIZE_TREE = (30, 10)
FIGSIZE_IMPORTANCE = (10, 12)
# ----------------------------


def load_model(path: str) -> xgb.Booster:
    model = xgb.Booster()
    model.load_model(path)
    print(f"Modelo cargado desde {path}")
    print("Cantidad de árboles:", model.num_boosted_rounds())
    print("Features del modelo:", model.feature_names)
    return model


def plot_importance_and_save(model: xgb.Booster):
    fig, ax = plt.subplots(figsize=FIGSIZE_IMPORTANCE)
    xgb.plot_importance(model, max_num_features=MAX_IMPORTANT_FEATURES, ax=ax)
    plt.tight_layout()
    out = "feature_importance.png"
    plt.savefig(out, dpi=200)
    plt.close(fig)
    print(f"Importancia de variables guardada en: {out}")


def get_sorted_features(model: xgb.Booster):
    # importancia por "gain"
    score_dict = model.get_score(importance_type="gain")
    sorted_features = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)

    feature_names = model.feature_names

    def real_name(fx: str) -> str:
        # Manejar dos casos:
        # - Si fx es del tipo "f12" (índice), mapear al nombre real si feature_names existe.
        # - Si fx ya es el nombre real (ej. "C01S"), devolverlo tal cual.
        if isinstance(fx, str) and fx.startswith("f") and fx[1:].isdigit():
            idx = int(fx[1:])  # "f12" -> 12
            if feature_names is not None and 0 <= idx < len(feature_names):
                return feature_names[idx]
            # si no hay feature_names disponible, devolver el identificador original
            return fx
        else:
            return fx

    print("\nTop features (XGBoost index → nombre real → importancia):")
    for f, s in sorted_features[:10]:
        print(f"{f} → {real_name(f)} → {s}")

    # devolver también mapeadas a nombres reales
    top_real = [real_name(f) for f, _ in sorted_features]
    return sorted_features, top_real


def find_trees_using_features(model: xgb.Booster, important_real_features, top_n=5):
    dump = model.get_dump(with_stats=True)
    selected_trees = []

    # Nos quedamos con las top_n variables más importantes
    important_real_features = important_real_features[:top_n]

    print(f"\nBuscando árboles que usen estas variables: {important_real_features}")

    for i, tree in enumerate(dump):
        for var in important_real_features:
            if var in tree:
                selected_trees.append(i)
                break

    print("Árboles que usan al menos una de las top vars:", selected_trees)
    return selected_trees


def plot_selected_trees(model: xgb.Booster, tree_indices):
    for i in tree_indices:
        fig, ax = plt.subplots(figsize=FIGSIZE_TREE)
        xgb.plot_tree(model, num_trees=i, ax=ax)
        plt.tight_layout()
        fname = f"xgboost_tree_topvar_{i}.png"
        plt.savefig(fname, dpi=200)
        plt.close(fig)
        print(f"Guardado árbol {i} en {fname}")


if __name__ == "__main__":
    # 1) Cargar modelo
    model = load_model(MODEL_PATH)

    # 2) Gráfico de importancia
    plot_importance_and_save(model)

    # 3) Ranking de features
    sorted_feats, top_real_names = get_sorted_features(model)

    # 4) Buscar árboles que usen las top vars
    selected = find_trees_using_features(model, top_real_names, top_n=TOP_FEATURES_FOR_TREES)

    # 5) Graficar solo esos árboles
    plot_selected_trees(model, selected)


Modelo cargado desde model.json
Cantidad de árboles: 60
Features del modelo: ['REV328', 'BC01S', 'AT104S', 'REV112', 'AT34B', 'BC20S', 'ALL255', 'G051S', 'FI33S_LG', 'AGG1123', 'RI31S', 'MNPMAG01', 'RET84', 'ALL253', 'AGG324', 'AT09SF', 'RI201S', 'BALMAG03_BG', 'TRANBAL21', 'BALMAG01', 'TELCO_AGG9101', 'AT29S', 'REV84', 'AU51A', 'RLE907', 'LL34S', 'FU20S', 'SE21S', 'ALL231', 'BALMAG04', 'BI01S', 'BKC235', 'RI201S_BG', 'RE12S', 'RET112', 'OF01S', 'IN34S', 'LMD34S', 'TEL27S', 'PAYMNT02']
Importancia de variables guardada en: feature_importance.png

Top features (XGBoost index → nombre real → importancia):
BC01S → BC01S → 185.5720672607422
AT104S → AT104S → 146.81712341308594
REV328 → REV328 → 141.2540740966797
REV112 → REV112 → 101.2645492553711
G051S → G051S → 90.06824493408203
AT34B → AT34B → 80.34980773925781
REV84 → REV84 → 72.13993072509766
FI33S_LG → FI33S_LG → 52.731781005859375
RLE907 → RLE907 → 45.61186218261719
AT29S → AT29S → 40.91239547729492

Buscando árboles que usen estas 

/Users/patricialorenasarmientotagle/Env/ml-env/lib/python3.11/site-packages/xgboost/plotting.py:267: FutureWarning: The `num_trees` parameter is deprecated, use `tree_idx` insetad. 
  warnings.warn(


Guardado árbol 0 en xgboost_tree_topvar_0.png


/Users/patricialorenasarmientotagle/Env/ml-env/lib/python3.11/site-packages/xgboost/plotting.py:267: FutureWarning: The `num_trees` parameter is deprecated, use `tree_idx` insetad. 
  warnings.warn(


Guardado árbol 1 en xgboost_tree_topvar_1.png


...

Guardado árbol 57 en xgboost_tree_topvar_57.png


/Users/patricialorenasarmientotagle/Env/ml-env/lib/python3.11/site-packages/xgboost/plotting.py:267: FutureWarning: The `num_trees` parameter is deprecated, use `tree_idx` insetad. 
  warnings.warn(


Guardado árbol 58 en xgboost_tree_topvar_58.png


/Users/patricialorenasarmientotagle/Env/ml-env/lib/python3.11/site-packages/xgboost/plotting.py:267: FutureWarning: The `num_trees` parameter is deprecated, use `tree_idx` insetad. 
  warnings.warn(


Guardado árbol 59 en xgboost_tree_topvar_59.png
